<a href="https://colab.research.google.com/github/akhilmenonh/fcc_ml_with_python/blob/Main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__) '''

'# import libraries\ntry:\n  # %tensorflow_version only exists in Colab.\n  !pip install tf-nightly\nexcept Exception:\n  pass\nimport tensorflow as tf\nimport pandas as pd\nfrom tensorflow import keras\n!pip install tensorflow-datasets\nimport tensorflow_datasets as tfds\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nprint(tf.__version__) '

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.19.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-10-06 11:17:45--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2025-10-06 11:17:45 (64.7 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-10-06 11:17:46--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [ ]:
# Load and prepare data
train_data = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
test_data = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

# Convert labels to binary
train_data["label"] = train_data["label"].map({"ham": 0, "spam": 1})
test_data["label"] = test_data["label"].map({"ham": 0, "spam": 1})

# Vectorization
max_vocab_size = 1000
max_len = 100

vectorizer = TextVectorization(max_tokens=max_vocab_size, output_mode="int", output_sequence_length=max_len)
vectorizer.adapt(train_data["message"])

X_train = vectorizer(train_data["message"])
y_train = train_data["label"].values

X_test = vectorizer(test_data["message"])
y_test = test_data["label"].values

# Build model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_vocab_size, output_dim=16),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Handle class imbalance
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Train model with class weights
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=10, verbose=2, class_weight=class_weights_dict)


Epoch 1/10
131/131 - 2s - 18ms/step - accuracy: 0.7758 - loss: 0.6749 - val_accuracy: 0.8915 - val_loss: 0.6263
Epoch 2/10
131/131 - 1s - 8ms/step - accuracy: 0.8849 - loss: 0.5553 - val_accuracy: 0.9124 - val_loss: 0.3317
Epoch 3/10
131/131 - 1s - 8ms/step - accuracy: 0.8940 - loss: 0.3748 - val_accuracy: 0.9009 - val_loss: 0.3019
Epoch 4/10
131/131 - 1s - 8ms/step - accuracy: 0.9098 - loss: 0.2749 - val_accuracy: 0.9152 - val_loss: 0.2417
Epoch 5/10
131/131 - 1s - 11ms/step - accuracy: 0.9201 - loss: 0.2323 - val_accuracy: 0.9382 - val_loss: 0.1735
Epoch 6/10
131/131 - 1s - 11ms/step - accuracy: 0.9375 - loss: 0.1972 - val_accuracy: 0.9246 - val_loss: 0.2233
Epoch 7/10
131/131 - 1s - 11ms/step - accuracy: 0.9478 - loss: 0.1701 - val_accuracy: 0.9318 - val_loss: 0.2227
Epoch 8/10
131/131 - 2s - 16ms/step - accuracy: 0.9555 - loss: 0.1495 - val_accuracy: 0.9325 - val_loss: 0.2246
Epoch 9/10
131/131 - 1s - 9ms/step - accuracy: 0.9617 - loss: 0.1384 - val_accuracy: 0.9662 - val_loss: 0.1

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text, threshold=0.4):
    input_text = tf.convert_to_tensor([pred_text])
    vectorized = vectorizer(input_text)
    prediction = model.predict(vectorized)[0][0]
    label = "spam" if prediction > threshold else "ham"
    return [float(prediction), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
[0.14315567910671234, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
You passed the challenge. Great job!
